# 🔧 Unified LaMa + ZITS Video Subtitle Inpainting Pipeline
This notebook demonstrates a unified pipeline for removing burnt-in subtitles using LaMa (static frame inpainting) and ZITS (video-aware inpainting).

✨ **Features:**
- 🚀 Auto GPU/CPU detection
- 📦 Docker containerization
- 🔄 Flexible local/cloud deployment
- 🎯 Production-ready pipeline

## 🔍 Step 0: Environment Detection
First, let's detect the available hardware and configure the environment accordingly.

In [ ]:
import torch
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from pathlib import Path

# Try to import AWS libraries
try:
    import boto3
    from botocore.exceptions import ClientError, NoCredentialsError
    aws_available = True
except ImportError:
    aws_available = False

# Environment detection
def detect_environment():
    """Detect and configure the runtime environment."""
    config = {
        'gpu_available': torch.cuda.is_available(),
        'gpu_count': torch.cuda.device_count() if torch.cuda.is_available() else 0,
        'device': 'cuda' if torch.cuda.is_available() else 'cpu',
        'in_docker': os.path.exists('/.dockerenv'),
        'in_colab': 'COLAB_GPU' in os.environ,
        'aws_available': aws_available,
        'aws_configured': False
    }
    
    if config['gpu_available']:
        config['gpu_name'] = torch.cuda.get_device_name(0)
        config['gpu_memory'] = torch.cuda.get_device_properties(0).total_memory / 1024**3
    
    # Check AWS configuration
    if aws_available:
        try:
            s3_client = boto3.client('s3')
            s3_client.list_buckets()
            config['aws_configured'] = True
        except (NoCredentialsError, ClientError):
            config['aws_configured'] = False
    
    return config

# Detect environment
env_config = detect_environment()

print("🔍 Environment Detection:")
print(f"   GPU Available: {env_config['gpu_available']}")
print(f"   Device: {env_config['device']}")
print(f"   Docker: {env_config['in_docker']}")
print(f"   Colab: {env_config['in_colab']}")
print(f"   AWS Libraries: {env_config['aws_available']}")
print(f"   AWS Configured: {env_config['aws_configured']}")

if env_config['gpu_available']:
    print(f"   GPU: {env_config['gpu_name']} ({env_config['gpu_memory']:.1f} GB)")
    print("   🚀 GPU acceleration enabled")
else:
    print("   💻 Running in CPU mode")

if env_config['aws_configured']:
    print("   ☁️  AWS S3 access configured")
elif env_config['aws_available']:
    print("   ⚠️  AWS libraries available but not configured")
else:
    print("   📦 Install boto3 for AWS/S3 support")

print()
print("✅ Environment configured successfully")

## 📁 Step 1: Setup and Data Generation
Configure directories and generate synthetic test data with subtitles and masks.

In [ ]:
# Configuration
CONFIG = {
    'input_dir': '../input_video',
    'mask_dir': '../masks',
    'lama_output_dir': '../lama_output',
    'zits_output_dir': '../zits_output',
    'frame_count': 30,  # More frames for better testing
    'frame_size': (256, 256),
    'subtitle_region': (0, 205, 256, 256),  # Bottom portion for subtitles
    # AWS/S3 Configuration
    's3_input_video': os.environ.get('S3_INPUT_VIDEO', ''),
    's3_output_location': os.environ.get('S3_OUTPUT_LOCATION', ''),
    'use_synthetic_data': os.environ.get('USE_SYNTHETIC_DATA', 'true').lower() == 'true'
}

# Create directories
for dir_path in [CONFIG['input_dir'], CONFIG['mask_dir'], CONFIG['lama_output_dir'], CONFIG['zits_output_dir']]:
    os.makedirs(dir_path, exist_ok=True)
    print(f"📁 Created directory: {dir_path}")

print(f"\n🔧 Pipeline Configuration:")
print(f"   S3 Input: {CONFIG['s3_input_video'] or 'Not specified'}")
print(f"   S3 Output: {CONFIG['s3_output_location'] or 'Not specified'}")
print(f"   Use Synthetic: {CONFIG['use_synthetic_data']}")

# Handle input data based on configuration
if CONFIG['s3_input_video'] and env_config['aws_configured']:
    print(f"\n📥 Processing S3 input video: {CONFIG['s3_input_video']}")
    
    if aws_available:
        try:
            # Import S3 handler functionality
            s3_client = boto3.client('s3')
            
            # Parse S3 URL
            s3_url = CONFIG['s3_input_video']
            if not s3_url.startswith('s3://'):
                raise ValueError("S3 URL must start with 's3://'")
            
            path = s3_url[5:]  # Remove 's3://'
            parts = path.split('/', 1)
            bucket = parts[0]
            key = parts[1]
            
            # Download video
            temp_video = "/tmp/input_video.mp4"
            print(f"   Downloading from S3...")
            s3_client.download_file(bucket, key, temp_video)
            
            # Extract frames using OpenCV
            cap = cv2.VideoCapture(temp_video)
            frame_count = 0
            
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                
                # Resize frame
                frame_resized = cv2.resize(frame, CONFIG['frame_size'])
                
                # Save frame
                frame_path = f"{CONFIG['input_dir']}/frame_{frame_count+1:05d}.png"
                cv2.imwrite(frame_path, frame_resized)
                frame_count += 1
            
            cap.release()
            
            # Generate masks for all frames (assume bottom subtitle region)
            for i in range(frame_count):
                mask = np.zeros(CONFIG['frame_size'], dtype=np.uint8)
                x1, y1, x2, y2 = CONFIG['subtitle_region']
                cv2.rectangle(mask, (x1, y1), (x2, y2), 255, -1)
                cv2.imwrite(f"{CONFIG['mask_dir']}/mask_{i+1:05d}.png", mask)
            
            print(f"✅ Processed S3 video: {frame_count} frames extracted")
            CONFIG['frame_count'] = frame_count
            
        except Exception as e:
            print(f"❌ S3 processing failed: {str(e)}")
            print("   Falling back to synthetic data generation...")
            CONFIG['use_synthetic_data'] = True
    else:
        print("❌ AWS libraries not available")
        CONFIG['use_synthetic_data'] = True

elif CONFIG['use_synthetic_data']:
    print(f"\n🎬 Generating {CONFIG['frame_count']} synthetic test frames...")

    # Generate synthetic frames with subtitles
    for i in range(CONFIG['frame_count']):
        # Create frame with animated background
        img = np.full((*CONFIG['frame_size'], 3), 255, dtype=np.uint8)
        
        # Add animated gradient effect
        phase = i / CONFIG['frame_count'] * 2 * np.pi
        for y in range(CONFIG['frame_size'][0]):
            for x in range(CONFIG['frame_size'][1]):
                # Create moving gradient
                r = int(128 + 127 * np.sin(phase + x/30))
                g = int(128 + 127 * np.sin(phase + y/30 + np.pi/3))
                b = int(128 + 127 * np.sin(phase + (x+y)/40 + 2*np.pi/3))
                img[y, x] = [min(255, max(0, r)), min(255, max(0, g)), min(255, max(0, b))]
        
        # Add main content text
        cv2.putText(img, f"Test Video Frame {i+1:02d}", (20, 50), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        cv2.putText(img, f"Sample Content", (20, 80), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)
        
        # Add moving object for realistic content
        center_x = int(128 + 50 * np.sin(i/5))
        center_y = int(120 + 30 * np.cos(i/3))
        cv2.circle(img, (center_x, center_y), 15, (0, 255, 255), -1)
        
        # Add subtitle (to be removed) with variation
        subtitle_texts = [
            "This is a sample subtitle",
            "Subtitle text to remove", 
            "Burnt-in subtitle example",
            f"Frame {i+1} subtitle",
            "AI will remove this text"
        ]
        subtitle = subtitle_texts[i % len(subtitle_texts)]
        
        # Add subtitle background box
        text_size = cv2.getTextSize(subtitle, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
        cv2.rectangle(img, (10, 215), (text_size[0] + 20, 245), (0, 0, 0), -1)
        
        # Add subtitle text
        cv2.putText(img, subtitle, (15, 235), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
        # Save frame
        cv2.imwrite(f"{CONFIG['input_dir']}/frame_{i+1:05d}.png", img)

        # Create corresponding mask for subtitle area
        mask = np.zeros(CONFIG['frame_size'], dtype=np.uint8)
        x1, y1, x2, y2 = CONFIG['subtitle_region']
        cv2.rectangle(mask, (x1, y1), (x2, y2), 255, -1)
        
        # Save mask
        cv2.imwrite(f"{CONFIG['mask_dir']}/mask_{i+1:05d}.png", mask)

    print(f"✅ Generated {CONFIG['frame_count']} synthetic frames and masks")

else:
    print("⚠️  No input specified and synthetic data disabled")
    print("   Looking for existing data in directories...")

print(f"   📁 Frames: {CONFIG['input_dir']}")
print(f"   🎭 Masks: {CONFIG['mask_dir']}")

## 👁️ Step 2: Visualize Generated Data
Let's preview the generated frames and masks to ensure they're correct.

In [ ]:
# Visualize sample data
sample_frame = cv2.imread(f"{CONFIG['input_dir']}/frame_000.png")
sample_mask = cv2.imread(f"{CONFIG['mask_dir']}/mask_000.png", 0)

if sample_frame is not None and sample_mask is not None:
    # Convert BGR to RGB for matplotlib
    sample_frame_rgb = cv2.cvtColor(sample_frame, cv2.COLOR_BGR2RGB)
    
    # Create visualization
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Original frame
    axes[0].imshow(sample_frame_rgb)
    axes[0].set_title('🎬 Original Frame\n(with subtitle)')
    axes[0].axis('off')
    
    # Mask
    axes[1].imshow(sample_mask, cmap='gray')
    axes[1].set_title('🎭 Subtitle Mask\n(white = remove)')
    axes[1].axis('off')
    
    # Overlay
    overlay = sample_frame_rgb.copy()
    overlay[sample_mask > 0] = [255, 0, 0]  # Red overlay on subtitle area
    combined = cv2.addWeighted(sample_frame_rgb, 0.7, overlay, 0.3, 0)
    axes[2].imshow(combined)
    axes[2].set_title('🔍 Frame + Mask Overlay\n(red = area to inpaint)')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Sample data visualization complete")
else:
    print("❌ Could not load sample data for visualization")

## 🔧 Step 3: Pipeline Instructions
Based on your environment, here are the next steps to complete the pipeline.

In [ ]:
print("🚀 Pipeline Instructions:")
print("="*50)

if env_config['in_docker']:
    print("🐳 Running in Docker - Pipeline will execute automatically!")
    print("   The entrypoint script will handle LaMa and ZITS++ execution.")
elif env_config['in_colab']:
    print("☁️  Running in Google Colab:")
    print("   1. Install dependencies: !pip install boto3 lama-cleaner")
    print("   2. Clone repositories manually")
    print("   3. Run commands below")
elif env_config['aws_configured']:
    print("☁️  Running with AWS support:")
    print("   ? Recommended: Use AWS Docker mode")
    print("   cd .. && ./test_pipeline.sh aws gpu s3://bucket/video.mp4 s3://bucket/output/")
    print()
    print("🔧 Alternative: Manual AWS setup")
    print("   Follow the commands below with S3 integration")
else:
    print("?💻 Running locally - Choose your preferred method:")
    print()
    print("🐳 Option 1: Docker (Recommended)")
    print("   cd .. && ./test_pipeline.sh docker")
    print()
    print("☁️  Option 2: AWS with S3")
    print("   # Set AWS credentials first")
    print("   export AWS_ACCESS_KEY_ID=your_key")
    print("   export AWS_SECRET_ACCESS_KEY=your_secret")
    print("   cd .. && ./test_pipeline.sh aws gpu s3://bucket/video.mp4 s3://bucket/output/")
    print()
    print("🔧 Option 3: Manual Setup")
    print("   Follow the commands below:")

print()
print("📋 Manual Commands:")
print()
print("🎨 LaMa Inpainting:")
print("   git clone https://github.com/advimman/lama.git")
print("   cd lama && pip install -r requirements.txt")
weight_path = "big-lama" if env_config['gpu_available'] else "PATH_TO_LAMA_WEIGHTS"
print(f"   python bin/predict.py model.path={weight_path} \\")
print(f"       indir={CONFIG['input_dir']} \\")
print(f"       maskdir={CONFIG['mask_dir']} \\")
print(f"       outdir={CONFIG['lama_output_dir']}")
print()
print("🎬 ZITS++ Temporal Refinement:")
print("   git clone https://github.com/DQiaole/ZITS_plus_plus.git")
print("   cd ZITS_plus_plus && pip install -r requirements.txt")
print(f"   python test_video.py \\")
print(f"       --video_root {CONFIG['input_dir']} \\")
print(f"       --mask_root {CONFIG['mask_dir']} \\")
print(f"       --inpainted_root {CONFIG['lama_output_dir']} \\")
print(f"       --output_root {CONFIG['zits_output_dir']}")
print()
print("🎞️  Final Video Assembly:")
print(f"   ffmpeg -framerate 30 -i {CONFIG['zits_output_dir']}/frame_%05d.png \\")
print("       -c:v libx264 -pix_fmt yuv420p output_clean.mp4")
print()

print("✨ AWS/S3 Integration:")
if env_config['aws_configured']:
    print("   ✅ AWS configured - S3 operations available")
    if CONFIG['s3_input_video']:
        print(f"   📥 S3 Input: {CONFIG['s3_input_video']}")
    if CONFIG['s3_output_location']:
        print(f"   📤 S3 Output: {CONFIG['s3_output_location']}")
else:
    print("   🔧 To enable S3 support:")
    print("     pip install boto3")
    print("     export AWS_ACCESS_KEY_ID=your_key")
    print("     export AWS_SECRET_ACCESS_KEY=your_secret")

print()
print("✨ Deployment Options:")
if env_config['gpu_available']:
    print("   🚀 GPU detected - processing will be much faster!")
    print("   📥 LaMa weights will be auto-downloaded in Docker mode")
    print("   💡 Use GPU instances (p3, p4, g4) on AWS for best performance")
else:
    print("   💻 CPU mode - processing will take longer but still work")
    print("   📥 Download LaMa weights manually for local setup")
    print("   💡 Use compute-optimized instances (c5, c6i) on AWS")

print(f"   📁 All output will be in: {CONFIG['zits_output_dir']}")
print("   🔄 Rerun this notebook to generate new test data")
print("   ☁️  Use AWS for production workloads with large videos")

## 📊 Step 4: Status Check
Monitor the pipeline progress and check outputs.

In [ ]:
def check_pipeline_status():
    """Check the status of pipeline outputs."""
    status = {
        'input_frames': len([f for f in os.listdir(CONFIG['input_dir']) if f.endswith('.png')]) if os.path.exists(CONFIG['input_dir']) else 0,
        'masks': len([f for f in os.listdir(CONFIG['mask_dir']) if f.endswith('.png')]) if os.path.exists(CONFIG['mask_dir']) else 0,
        'lama_output': len([f for f in os.listdir(CONFIG['lama_output_dir']) if f.endswith('.png')]) if os.path.exists(CONFIG['lama_output_dir']) else 0,
        'zits_output': len([f for f in os.listdir(CONFIG['zits_output_dir']) if f.endswith('.png')]) if os.path.exists(CONFIG['zits_output_dir']) else 0,
    }
    
    print("📊 Pipeline Status:")
    print("="*30)
    print(f"📁 Input frames: {status['input_frames']} {'✅' if status['input_frames'] > 0 else '❌'}")
    print(f"🎭 Masks: {status['masks']} {'✅' if status['masks'] > 0 else '❌'}")
    print(f"🎨 LaMa output: {status['lama_output']} {'✅' if status['lama_output'] > 0 else '⏳'}")
    print(f"🎬 ZITS output: {status['zits_output']} {'✅' if status['zits_output'] > 0 else '⏳'}")
    
    if status['zits_output'] > 0:
        print("\n🎉 Pipeline completed successfully!")
        print(f"   Final results: {CONFIG['zits_output_dir']}")
        return True
    elif status['lama_output'] > 0:
        print("\n⏳ LaMa completed, ZITS++ pending...")
        return False
    elif status['input_frames'] > 0:
        print("\n⏳ Test data ready, processing pending...")
        return False
    else:
        print("\n❌ No data found - run the generation cells above")
        return False

# Check current status
pipeline_complete = check_pipeline_status()

if pipeline_complete:
    print("\n🎯 Next steps:")
    print("   1. Review output frames")
    print("   2. Create final video with FFmpeg")
    print("   3. Process your own video data")